# Warm Start Testing

In [2]:
import numpy as np
import gurobipy as gp
import math
import sympy
import contextlib
import time
import os
from utils import result, METHODS, INF, EPS
from mps_reader_preprocessor import read_mps_preprocess
from polyhedral_model import PolyhedralModel
from polyhedron import Polyhedron
import network as net

In [3]:
def num_aft_dec(number): 
    # Convert number to string
    number_str = str(number)
    
    # Split string at the decimal point
    if '.' in number_str:
        # Get the part after the decimal point
        decimal_part = number_str.split('.')[1]
        
        # Remove trailing zeros
        decimal_part = decimal_part.rstrip('0')
        
        # Return the length of the remaining decimal part
        return len(decimal_part)
    else:
        # No decimal point in the number
        return 0
    

## Read in Polyhedron P

In [4]:
problem_dir = 'netlib_lp_subset'
# problem = 'kb2'
# problem = 'maros'
# problem = 'pilot4'
problem = 'shell'
# problem = 'adlittle'
# problem = 'angela_test_prob'
# problem = 'network_ex'
# problem = 'gfrd-pnc'
# problem = 'modszk1'
# problem = 'cycle'
# problem = 'cre-c'
# problem = 'scsd8'
### problems in standard form:
# 'bandm'
# fit1p contains upper bounds
# grow15 contains upper bounds
# grow 22 contains upper bounds
# grow7 contains upper bounds
# ken-07 contains upper bounds
# modszk1 contains free variables
# 'qap08'
# 'scsd1'
# 'scsd6'
# 'scsd8'

mps_fn=os.path.join(problem_dir, problem)
results_dir='results'
max_time=300
sd_method='dual_simplex'
reset=False,
partition_polytope=False
n=0
k=0
spindle=False
spindle_dim=0
n_cone_facets=0
n_parallel_facets=0

### Build Initial Polyhedron from file
print(f'Reading {mps_fn}...')
c, B, d, A, b = read_mps_preprocess(mps_fn)
print('Building polyhedron...')
P = Polyhedron(B, d, A, b, c)

print('Finding feasible solution...')
x_feasible, vbasis, cbasis = P.find_feasible_solution(verbose=False)

Reading netlib_lp_subset\shell...
Building polyhedron...
Problem size: n = 1775,  m_B = 1644,  m_A = 784
Finding feasible solution...
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-01


In [5]:
eq_cont_sat = np.all(np.isclose(np.dot(A, np.array(x_feasible)),b))
ineq_cont_sat = np.all(np.dot(B, np.array(x_feasible)) <=d)
print(f'equality constraints satisfied by feaisble soln: {eq_cont_sat}')
print(f'inequality constraints satisfied by feaisble soln: {ineq_cont_sat}')

equality constraints satisfied by feaisble soln: True
inequality constraints satisfied by feaisble soln: True


In [6]:
x = x_feasible
c=None
verbose=False
method='dual_simplex'
reset=False
max_time=300
first_warm_start=None
save_first_steps=0
problem_name=''

In [9]:
if c is not None:
    P.set_objective(c)
 
t0 = time.time()
x_current = x
if save_first_steps:
    np.save('solutions/{}_0.npy'.format(problem_name), x_current)      
active_inds = P.get_active_constraints(x_current)
init_inds = active_inds
    
pm = P.build_polyhedral_model(active_inds=active_inds, method=method)

if first_warm_start is not None:
    print('Using custom warm start')
    pm.set_solution(first_warm_start)
t1 = time.time()
build_time = t1 - t0
print('Polyhedral model build time: {}'.format(build_time))
    
sub_times = {'sd': [], 'step': [], 'solve': [], 'phase_times': []}    
descent_circuits = []
obj_values = []
step_sizes = []
iter_times = []
simplex_iters = []
iteration = 0
obj_value = P.c.dot(x_current)
num_dec_places = Polyhedron.num_aft_dec(obj_value)
obj_values.append(obj_value)
t2 = time.time()
iter_times.append(t2 - t1)

####get edge for initial circuit direction here#########
x_feasible_2= P.second_vert(x_current, obj_value, num_dec_places, verbose=False, vbasis = vbasis, cbasis = cbasis)
edge = np.array(x_feasible_2) - np.array(x_current)
B_edge = np.dot(B,np.array(edge))
norm = np.linalg.norm(np.array(B_edge),1)
normed_B_edge = B_edge/(norm)
init_y_pos = []
init_y_neg = []
for entry in normed_B_edge:
    if entry > 0:
        init_y_pos.append(entry)
        init_y_neg.append(0)
    else:
        init_y_pos.append(0)
        init_y_neg.append(-entry)
########################################################

# # compute steepest-descent direction
descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(y_pos = init_y_pos, y_neg = init_y_neg, 
                                                                                                        verbose=verbose)
# print(f'number of nonzero entries in first descent direction: {np.count_nonzero(np.array(test_dir))}')
simplex_iters.append(num_steps)
sub_times['solve'].append(solve_time)
sub_times['phase_times'].append(phase_times)

t3 = time.time()
sub_times['sd'].append(t3 - t2)

    
while abs(steepness) > EPS:
        
    t3 = time.time()
    if reset:
        pm.reset()
        
    # take maximal step
    x_current, alpha, active_inds = P.take_maximal_step(descent_direction, y_pos, y_neg)  
        
    if iteration % 50 == 0 or iteration == 1:
        print('\nIteration {}'.format(iteration))
        print('Objective: {}'.format(obj_value))
        print('Steepness: {}'.format(steepness))
        print('Step length: {}'.format(alpha))
        
    t4 = time.time()
    obj_value = P.c.dot(x_current)
    obj_values.append(obj_value)
    iter_times.append(t4 - t1)
    sub_times['step'].append(t4 - t3) 
    descent_circuits.append(descent_direction)
    step_sizes.append(alpha)     
                
    if math.isinf(alpha):
        # problem is unbounded
        result(status=1, circuits=descent_circuits, steps=step_sizes)
        
    # compute steepest-descent direction
    pm.set_active_inds(active_inds)
    descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(verbose=verbose)
        
    t5 = time.time()
    sub_times['sd'].append(t5 - t4)
    sub_times['solve'].append(solve_time)
    sub_times['phase_times'].append(phase_times)
    simplex_iters.append(num_steps)
        
    iteration += 1
    current_time = t5 - t1
    if current_time > max_time:
        result(status=2)
    if iteration <= save_first_steps:
        np.save('solutions/{}_{}.npy'.format(problem_name, iteration), x_current)

t6 = time.time()
total_time = t6 - t1   
print('Total time for steepest-descent scheme: {}'.format(total_time))

Building polyhedral model. Solve method: dual_simplex
Set parameter Method to value 1
Polyhedral model built!
Polyhedral model build time: 2.9860870838165283
other warm start
Model is feasible or optimization was successful.

Iteration 0
Objective: 1268012500.0
Steepness: -173.0
Step length: 44315.00000000001

Iteration 1
Objective: 1260346005.0
Steepness: -159.0
Step length: 518.9999999999999

Iteration 50
Objective: 1233330348.0
Steepness: -24.833333333333364
Step length: 2760.000000000043

Iteration 100
Objective: 1213984581.0
Steepness: -9.200000000000001
Step length: 5055.0

Iteration 150
Objective: 1208983019.0
Steepness: -1.8888888888888857
Step length: 198.00000000000077
Total time for steepest-descent scheme: 5.167969703674316


In [8]:
normed_B_edge

array([ 0.01813688, -0.00105307,  0.        , ...,  0.        ,
        0.        ,  0.        ])

In [ ]:
np.array_equal(-P.B,np.eye(P.m_B))

In [ ]:
print(obj_values[-1])
# print(x_current)